In [2]:
!pip install folium

  Using cached folium-0.20.0-py2.py3-none-any.whl.metadata (4.2 kB)
  Using cached branca-0.8.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached xyzservices-2025.11.0-py3-none-any.whl.metadata (4.3 kB)
Using cached folium-0.20.0-py2.py3-none-any.whl (113 kB)
Using cached branca-0.8.2-py3-none-any.whl (26 kB)
Using cached xyzservices-2025.11.0-py3-none-any.whl (93 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [folium]


In [4]:
import folium
import pandas as pd
import numpy as np
import json
from folium.plugins import FloatImage
import warnings
warnings.filterwarnings('ignore')

# ===== 1. 서울시 자치구 리스트 =====
SEOUL_DISTRICTS = [
    '종로구', '중구', '용산구', '성동구', '광진구',
    '동대문구', '중랑구', '성북구', '강북구', '도봉구',
    '노원구', '은평구', '서대문구', '마포구', '양천구',
    '강서구', '구로구', '금천구', '영등포구', '동작구',
    '관악구', '서초구', '강남구', '송파구', '강동구'
]

# ===== 2. 위험도 예측 함수 (Step 4 예측기 로직 통합) =====
def calculate_risk_probability(district, time_slot, crime_type, location_type):
    """
    특정 시나리오에 대한 자치구별 위험 확률 계산
    
    Parameters:
    - district: 자치구 이름 (예: '강남구')
    - time_slot: 시간대 (예: '21시-24시')
    - crime_type: 범죄 유형 (예: '폭력')
    - location_type: 장소 유형 (예: '다중이용시설')
    
    Returns:
    - risk_probability: 0~1 사이의 위험 확률
    """
    
    # 실제 모델이 없는 경우를 위한 시뮬레이션 로직
    # 실제로는 Step 3의 훈련된 모델을 로드하여 예측해야 합니다
    
    # 기본 위험도 설정
    base_risk = 0.3
    
    # 자치구별 가중치 (실제 범죄 통계 기반 예시)
    district_weights = {
        '강남구': 1.4, '강서구': 1.3, '송파구': 1.2, '영등포구': 1.3,
        '관악구': 1.2, '종로구': 1.1, '중구': 1.2, '구로구': 1.25,
        '용산구': 1.1, '서초구': 1.0, '마포구': 1.15, '동대문구': 1.1,
        '성동구': 1.0, '광진구': 1.0, '중랑구': 1.0, '성북구': 1.0,
        '강북구': 1.05, '도봉구': 0.9, '노원구': 1.0, '은평구': 0.95,
        '서대문구': 1.0, '양천구': 0.9, '금천구': 1.1, '동작구': 1.0,
        '강동구': 0.95
    }
    
    # 시간대별 가중치
    time_weights = {
        '00시-03시': 1.5, '03시-06시': 1.2, '06시-09시': 0.8,
        '09시-12시': 0.7, '12시-15시': 0.8, '15시-18시': 0.9,
        '18시-21시': 1.1, '21시-24시': 1.4
    }
    
    # 범죄 유형별 가중치
    crime_weights = {
        '폭력': 1.3, '절도': 1.2, '성범죄': 1.4, '마약': 1.1, '기타': 1.0
    }
    
    # 장소 유형별 가중치
    location_weights = {
        '다중이용시설': 1.3, '주거지역': 1.1, '상업지역': 1.2,
        '유흥가': 1.5, '공원': 1.1, '교통시설': 1.2, '기타': 1.0
    }
    
    # 위험도 계산
    risk = base_risk
    risk *= district_weights.get(district, 1.0)
    risk *= time_weights.get(time_slot, 1.0)
    risk *= crime_weights.get(crime_type, 1.0)
    risk *= location_weights.get(location_type, 1.0)
    
    # 약간의 랜덤 노이즈 추가 (현실적인 변동성)
    risk *= np.random.uniform(0.9, 1.1)
    
    # 0~1 범위로 정규화
    risk = min(max(risk, 0.0), 1.0)
    
    return risk

# ===== 3. 전체 자치구 위험도 데이터 생성 =====
def generate_risk_data(time_slot, crime_type, location_type):
    """
    모든 서울시 자치구에 대한 위험도 데이터 생성
    """
    risk_data = []
    
    for district in SEOUL_DISTRICTS:
        risk_prob = calculate_risk_probability(
            district, time_slot, crime_type, location_type
        )
        
        risk_data.append({
            'district': district,
            'risk_probability': risk_prob,
            'risk_percentage': round(risk_prob * 100, 2)
        })
    
    return pd.DataFrame(risk_data)

# ===== 4. Folium 지도 생성 함수 =====
def create_risk_map(risk_df, time_slot, crime_type, location_type, 
                    geojson_path='/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/seoul_municipalities_geo_simple.txt',
                    output_file='seoul_crime_risk_map.html'):
    """
    위험도 Choropleth 지도 생성
    
    Parameters:
    - risk_df: 위험도 데이터프레임
    - time_slot, crime_type, location_type: 시나리오 정보
    - geojson_path: 서울시 자치구 GeoJSON 파일 경로
    - output_file: 출력 HTML 파일명
    """
    
    # 서울시 중심 좌표
    seoul_center = [37.5665, 126.9780]
    
    # Folium 지도 생성
    m = folium.Map(
        location=seoul_center,
        zoom_start=11,
        tiles='CartoDB positron',
        attr='Crime Risk Analysis'
    )
    
    # GeoJSON 데이터 로드 시도
    try:
        with open(geojson_path, 'r', encoding='utf-8') as f:
            geojson_data = json.load(f)
    except FileNotFoundError:
        print(f"⚠️  GeoJSON 파일을 찾을 수 없습니다: {geojson_path}")
        print("    샘플 GeoJSON을 생성합니다...")
        geojson_data = create_sample_geojson()
    
    # Choropleth 레이어 추가
    choropleth = folium.Choropleth(
        geo_data=geojson_data,
        name='위험도',
        data=risk_df,
        columns=['district', 'risk_probability'],
        key_on='feature.properties.name',  # GeoJSON의 자치구 이름 속성
        fill_color='YlOrRd',  # 노란색-주황색-빨간색
        fill_opacity=0.7,
        line_opacity=0.3,
        legend_name='범죄 위험 확률',
        nan_fill_color='lightgray',
        nan_fill_opacity=0.4,
    ).add_to(m)
    
    # 툴팁 추가 (마우스 오버 시 표시)
    choropleth.geojson.add_child(
        folium.features.GeoJsonTooltip(
            fields=['name'],
            aliases=['자치구: '],
            style=("background-color: white; color: #333333; "
                   "font-family: arial; font-size: 12px; padding: 10px;")
        )
    )
    
    # 각 자치구별 팝업 추가
    for idx, row in risk_df.iterrows():
        district = row['district']
        risk_pct = row['risk_percentage']
        
        # 위험도에 따른 색상 결정
        if risk_pct >= 70:
            color = 'red'
            risk_level = '매우 높음'
        elif risk_pct >= 50:
            color = 'orange'
            risk_level = '높음'
        elif risk_pct >= 30:
            color = 'yellow'
            risk_level = '보통'
        else:
            color = 'green'
            risk_level = '낮음'
        
        # 팝업 HTML 생성
        popup_html = f"""
        <div style="font-family: Arial; width: 200px;">
            <h4 style="margin: 0; color: #2c3e50;">{district}</h4>
            <hr style="margin: 5px 0;">
            <p style="margin: 5px 0;">
                <b>위험도:</b> <span style="color: {color}; font-weight: bold;">
                {risk_pct}%</span>
            </p>
            <p style="margin: 5px 0;">
                <b>위험 수준:</b> <span style="color: {color};">{risk_level}</span>
            </p>
            <hr style="margin: 5px 0;">
            <p style="margin: 3px 0; font-size: 11px; color: #7f8c8d;">
                시간: {time_slot}<br>
                범죄: {crime_type}<br>
                장소: {location_type}
            </p>
        </div>
        """
        
        # 자치구 중심 좌표 추정 (실제로는 GeoJSON에서 추출)
        # 간단한 예시로 서울 중심 기준 약간의 변화
        lat = seoul_center[0] + np.random.uniform(-0.1, 0.1)
        lon = seoul_center[1] + np.random.uniform(-0.1, 0.1)
        
        folium.Marker(
            location=[lat, lon],
            popup=folium.Popup(popup_html, max_width=250),
            icon=folium.Icon(color=color, icon='info-sign')
        ).add_to(m)
    
    # 제목 추가
    title_html = f'''
    <div style="position: fixed; 
                top: 10px; left: 50px; width: 600px; height: 90px; 
                background-color: white; border: 2px solid grey; 
                z-index: 9999; font-size: 14px; padding: 10px;
                box-shadow: 2px 2px 6px rgba(0,0,0,0.3);">
        <h3 style="margin: 0; color: #c0392b;">🚨 서울시 범죄 위험도 분석 지도</h3>
        <p style="margin: 5px 0; font-weight: bold;">
            📅 시간대: {time_slot} | 
            🔍 범죄유형: {crime_type} | 
            📍 장소: {location_type}
        </p>
        <p style="margin: 5px 0; font-size: 12px; color: #7f8c8d;">
            💡 자치구를 클릭하면 상세 정보를 확인할 수 있습니다.
        </p>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(title_html))
    
    # 레이어 컨트롤 추가
    folium.LayerControl().add_to(m)
    
    # HTML 파일로 저장
    m.save(output_file)
    print(f"✅ 지도가 생성되었습니다: {output_file}")
    
    return m

# ===== 5. 샘플 GeoJSON 생성 함수 =====
def create_sample_geojson():
    """
    GeoJSON 파일이 없을 경우 간단한 샘플 생성
    (실제 사용시에는 실제 서울시 자치구 경계 GeoJSON 필요)
    """
    features = []
    
    for i, district in enumerate(SEOUL_DISTRICTS):
        # 간단한 사각형 폴리곤 생성 (예시)
        lat_base = 37.4 + (i // 5) * 0.05
        lon_base = 126.8 + (i % 5) * 0.05
        
        feature = {
            "type": "Feature",
            "properties": {
                "name": district,
                "code": f"11{i+1:03d}"
            },
            "geometry": {
                "type": "Polygon",
                "coordinates": [[
                    [lon_base, lat_base],
                    [lon_base + 0.04, lat_base],
                    [lon_base + 0.04, lat_base + 0.04],
                    [lon_base, lat_base + 0.04],
                    [lon_base, lat_base]
                ]]
            }
        }
        features.append(feature)
    
    return {
        "type": "FeatureCollection",
        "features": features
    }

# ===== 6. 메인 실행 함수 =====
def main():
    """
    메인 실행 함수
    """
    print("=" * 60)
    print("🗺️  서울시 범죄 위험도 지도 시각화 시스템")
    print("=" * 60)
    
    # 시나리오 설정
    scenario = {
        'time_slot': '21시-24시',
        'crime_type': '폭력',
        'location_type': '다중이용시설'
    }
    
    print(f"\n📊 시나리오 설정:")
    print(f"   - 시간대: {scenario['time_slot']}")
    print(f"   - 범죄 유형: {scenario['crime_type']}")
    print(f"   - 장소 유형: {scenario['location_type']}")
    
    # 위험도 데이터 생성
    print("\n⚙️  자치구별 위험도 계산 중...")
    risk_df = generate_risk_data(**scenario)
    
    # 통계 출력
    print(f"\n📈 위험도 통계:")
    print(f"   - 평균 위험도: {risk_df['risk_percentage'].mean():.2f}%")
    print(f"   - 최대 위험도: {risk_df['risk_percentage'].max():.2f}% ({risk_df.loc[risk_df['risk_percentage'].idxmax(), 'district']})")
    print(f"   - 최소 위험도: {risk_df['risk_percentage'].min():.2f}% ({risk_df.loc[risk_df['risk_percentage'].idxmin(), 'district']})")
    
    # 위험도 상위 5개 자치구
    print(f"\n🔴 위험도 TOP 5:")
    top5 = risk_df.nlargest(5, 'risk_percentage')
    for idx, row in top5.iterrows():
        print(f"   {row['district']:8s}: {row['risk_percentage']:5.2f}%")
    
    # 지도 생성
    print(f"\n🗺️  Choropleth 지도 생성 중...")
    create_risk_map(
        risk_df,
        scenario['time_slot'],
        scenario['crime_type'],
        scenario['location_type'],
        output_file='seoul_crime_risk_map.html'
    )
    
    print(f"\n✨ 완료! 웹 브라우저에서 'seoul_crime_risk_map.html'을 열어보세요.")
    print("=" * 60)

# ===== 7. 다양한 시나리오 비교 함수 =====
def create_multiple_scenarios():
    """
    여러 시나리오를 비교하는 지도 생성
    """
    scenarios = [
        {'time_slot': '21시-24시', 'crime_type': '폭력', 'location_type': '유흥가'},
        {'time_slot': '00시-03시', 'crime_type': '절도', 'location_type': '주거지역'},
        {'time_slot': '18시-21시', 'crime_type': '성범죄', 'location_type': '공원'},
    ]
    
    for i, scenario in enumerate(scenarios, 1):
        print(f"\n시나리오 {i} 생성 중...")
        risk_df = generate_risk_data(**scenario)
        create_risk_map(
            risk_df,
            scenario['time_slot'],
            scenario['crime_type'],
            scenario['location_type'],
            output_file=f'scenario_{i}_map.html'
        )

if __name__ == "__main__":
    main()
    
    # 추가: 여러 시나리오 생성 (선택사항)
    # create_multiple_scenarios()

🗺️  서울시 범죄 위험도 지도 시각화 시스템

📊 시나리오 설정:
   - 시간대: 21시-24시
   - 범죄 유형: 폭력
   - 장소 유형: 다중이용시설

⚙️  자치구별 위험도 계산 중...

📈 위험도 통계:
   - 평균 위험도: 76.93%
   - 최대 위험도: 100.00% (강남구)
   - 최소 위험도: 61.80% (도봉구)

🔴 위험도 TOP 5:
   강남구     : 100.00%
   구로구     : 93.93%
   영등포구    : 91.43%
   중구      : 88.99%
   송파구     : 87.81%

🗺️  Choropleth 지도 생성 중...
✅ 지도가 생성되었습니다: seoul_crime_risk_map.html

✨ 완료! 웹 브라우저에서 'seoul_crime_risk_map.html'을 열어보세요.
